In [3]:
!pip install polars --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg/polars-0.20.16-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg


In [4]:
import polars as pl
import duckdb as dd
import os
from datetime import datetime
import tensorflow as tf
# import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1729917064.738073      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D1026 04:31:04.746121386      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1026 04:31:04.746136261      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1026 04:31:04.746139457      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1026 04:31:04.746141892      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1026 04:31:04.746144230      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

In [5]:
# detect TPUs
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1729917088.384763      13 service.cc:145] XLA service 0x57177d9bdff0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729917088.384824      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1729917088.384829      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1729917088.384832      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1729917088.384835      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1729917088.384838      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1729917088.384841      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1729917088.384843      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1729917088.384846      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [6]:
config_dict = {'train_csv_path': '/kaggle/input/child-mind-institute-problematic-internet-use/train.csv'}
config_dict['test_csv_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/test.csv'
config_dict['train_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/'
config_dict['test_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/'

In [7]:
train_data = pl.read_csv(config_dict['train_csv_path'], low_memory=True)
#train_data.describe()

In [5]:
#train_data.glimpse()
#train_data.columns

In [15]:
train_data.filter(pl.col('id')=='00115b9f')

id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,…,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
str,str,i64,i64,str,i64,str,f64,f64,f64,f64,i64,i64,i64,str,i64,i64,i64,str,i64,i64,f64,i64,f64,i64,i64,i64,f64,i64,f64,i64,f64,i64,str,i64,f64,f64,…,f64,f64,f64,f64,f64,str,f64,str,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,str,i64,i64
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,30.4041,16.779,58.9338,26.4798,45.9966,null,null,"""Winter""",2.451,"""Summer""",4,2,4,0,5,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1


In [16]:
train_data_id_list = pl.Series(train_data.select(pl.col('id'))).to_list()
len(train_data_id_list)

3960

In [17]:
parquet_file_name = "id={0}/*.parquet".format(train_data_id_list[0])
sample_parquet_file = config_dict['train_parquet_root_path']+parquet_file_name

parquet_sample_df = pl.read_parquet(sample_parquet_file)

FileNotFoundError: No such file or directory (os error 2): ...child-mind-institute-problematic-internet-use/series_train.parquet/id=00008ff9/*.parquet

In [11]:
parquet_sample_df.describe()

statistic,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT,id
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""count""",43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,"""43330"""
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0"""
"""mean""",21664.5,-0.316384,0.016009,-0.16789,0.047388,-10.580416,0.0,42.29631,4053.578857,5.0462e13,4.470182,3.0,53.201683,null
"""std""",12508.437919,0.453665,0.502702,0.58571,0.106351,42.94717,0.0,208.168976,112.404037,1.9428e13,1.931421,0.0,14.244915,null
"""min""",0.0,-1.746094,-2.905339,-1.048372,0.0,-89.833092,0.0,0.0,3824.0,5.5000e10,1.0,3.0,41.0,"""00115b9f"""
"""25%""",10832.0,-0.684193,-0.30987,-0.649974,0.006431,-41.543835,0.0,2.392968,4028.666748,3.6890e13,3.0,3.0,42.0,null
"""50%""",21665.0,-0.366849,0.024974,-0.245365,0.023638,-15.084804,0.0,6.926829,4070.0,5.3480e13,5.0,3.0,50.0,null
"""75%""",32497.0,-0.010677,0.400677,0.20474,0.04142,12.220962,0.0,15.0,4147.0,6.6410e13,6.0,3.0,53.0,null
"""max""",43329.0,1.507865,1.666354,1.546979,4.004276,89.751656,0.0,2633.25,4188.5,8.6110e13,7.0,3.0,85.0,"""00115b9f"""


In [13]:
parquet_sample_df

step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
u32,f32,f32,f32,f32,f32,f32,f32,f32,i64,i8,i8,f32
0,0.021536,0.022214,-1.02237,0.022853,-88.280762,0.0,53.0,4188.0,56940000000000,4,3,41.0
1,0.022005,0.022187,-1.01974,0.020231,-88.241707,0.0,51.666668,4188.166504,56945000000000,4,3,41.0
2,0.02224,0.022005,-1.019401,0.019893,-88.170067,0.0,50.333332,4188.333496,56950000000000,4,3,41.0
3,0.021589,0.022578,-1.018177,0.018667,-88.250031,0.0,50.5,4188.5,56955000000000,4,3,41.0
4,0.022005,0.023763,-1.014323,0.016848,-88.130775,0.0,33.166668,4181.0,57235000000000,4,3,41.0
…,…,…,…,…,…,…,…,…,…,…,…,…
43325,-0.008333,-0.02362,-1.006901,0.007224,-88.595741,0.0,0.0,3824.0,57580000000000,6,3,85.0
43326,0.04873,-0.076725,-0.953776,0.056839,-85.352219,0.0,0.0,3824.0,57640000000000,6,3,85.0
43327,0.38737,0.793151,-0.402214,0.069961,-24.097908,0.0,0.0,3824.0,57645000000000,6,3,85.0


In [30]:
def get_hours(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[0])

def get_mins(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[1])
    
def get_secs(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[2])

In [35]:
parquet_sample_df = parquet_sample_df.with_columns([
    #pl.lit('00115b9f').alias("id"),
    pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day"),
    pl.col("time_of_day").map_elements(get_mins, return_dtype=pl.Int32).alias("min_of_day"),
    pl.col("time_of_day").map_elements(get_secs, return_dtype=pl.Int32).alias("sec_of_day")
])

parquet_sample_df = parquet_sample_df.drop(['time_of_day'])

parquet_sample_df.head(10)

step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,weekday,quarter,relative_date_PCIAT,id,hour_of_day,min_of_day,sec_of_day
u32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,f32,str,i32,i32,i32
0,0.021536,0.022214,-1.02237,0.022853,-88.280762,0.0,53.0,4188.0,4,3,41.0,"""00115b9f""",18,40,0
1,0.022005,0.022187,-1.01974,0.020231,-88.241707,0.0,51.666668,4188.166504,4,3,41.0,"""00115b9f""",15,33,20
2,0.02224,0.022005,-1.019401,0.019893,-88.170067,0.0,50.333332,4188.333496,4,3,41.0,"""00115b9f""",12,26,40
3,0.021589,0.022578,-1.018177,0.018667,-88.250031,0.0,50.5,4188.5,4,3,41.0,"""00115b9f""",9,20,0
4,0.022005,0.023763,-1.014323,0.016848,-88.130775,0.0,33.166668,4181.0,4,3,41.0,"""00115b9f""",3,6,40
5,0.023281,0.025156,-1.015599,0.01668,-88.130775,0.0,31.333334,4181.0,4,3,41.0,"""00115b9f""",0,0,0
6,0.024818,0.027109,-1.015781,0.017497,-88.130775,0.0,29.5,4181.0,4,3,41.0,"""00115b9f""",20,53,20
7,0.022578,0.024766,-1.015026,0.015589,-88.14167,0.0,27.666666,4181.0,4,3,41.0,"""00115b9f""",17,46,40
8,0.022318,0.024792,-1.01599,0.016576,-88.130775,0.0,25.833334,4181.0,4,3,41.0,"""00115b9f""",14,40,0


In [36]:
train_data_sample = train_data.filter(pl.col('id')=='00115b9f')

sample_joined_features = dd.sql("select t1.*, t2.* exclude(id) \
from train_data_sample t1 join parquet_sample_df t2 on t1.id = t2.id").pl()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [37]:
sample_joined_features.head(10)

id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,…,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,weekday,quarter,relative_date_PCIAT,hour_of_day,min_of_day,sec_of_day
str,str,i64,i64,str,i64,str,f64,f64,f64,f64,i64,i64,i64,str,i64,i64,i64,str,i64,i64,f64,i64,f64,i64,i64,i64,f64,i64,f64,i64,f64,i64,str,i64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,str,i64,i64,u32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,f32,i32,i32,i32
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43329,0.792144,0.170486,-0.413108,0.063103,-16.771301,0.0,0.0,3824.0,6,3,85.0,5,46,40
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43328,0.801953,0.501589,-0.040937,0.045489,-2.113776,0.0,0.0,3824.0,6,3,85.0,8,53,20
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43327,0.38737,0.793151,-0.402214,0.069961,-24.097908,0.0,0.0,3824.0,6,3,85.0,12,0,0
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43326,0.04873,-0.076725,-0.953776,0.056839,-85.352219,0.0,0.0,3824.0,6,3,85.0,15,6,40
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43325,-0.008333,-0.02362,-1.006901,0.007224,-88.595741,0.0,0.0,3824.0,6,3,85.0,4,26,40
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43324,-0.008542,-0.023385,-1.006797,0.007117,-88.595741,0.0,0.0,3824.0,6,3,85.0,7,33,20
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43323,-0.005859,-0.024089,-1.007552,0.013274,-88.570198,0.0,0.0,3824.0,6,3,85.0,10,40,0
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292

In [38]:
sample_joined_features.shape

(43330, 97)

In [21]:

actigraphy_file_list = []
actigraphy_id_list = []

for dirname, _, filenames in os.walk('/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet'):
    for filename in filenames:
        full_parquet_file_path = os.path.join(dirname, filename)
        actigraphy_file_list.append(full_parquet_file_path)
        actigraphy_id_list.append(full_parquet_file_path.split('/')[-2].replace('id=',''))

In [22]:
print(len(actigraphy_file_list))
print(len(actigraphy_id_list))

996
996


In [20]:
print(actigraphy_file_list[10])
actigraphy_file_list[10].split('/')[-2].replace('id=','')

/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=15dbc929/part-0.parquet


'15dbc929'

In [23]:
parquet_sample_df = pl.read_parquet(actigraphy_file_list[10])
parquet_sample_df.head(100)

step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT,id
u32,f32,f32,f32,f32,f32,f32,f32,f32,i64,i8,i8,f32,str
0,-0.009687,1.000598,0.04817,0.003217,2.781722,0.0,8.333333,4190.0,38760000000000,1,1,342.0,"""15dbc929"""
1,-0.123873,-0.127452,-0.312257,0.104779,-19.540867,0.0,7.666667,4188.416504,38765000000000,1,1,342.0,"""15dbc929"""
2,-0.350496,-0.586732,0.216211,0.081689,14.9663,0.0,6.0,4186.833496,38770000000000,1,1,342.0,"""15dbc929"""
3,-0.043604,-0.491461,0.466575,0.075197,31.550388,0.0,7.888889,4185.25,38775000000000,1,1,342.0,"""15dbc929"""
4,0.406653,-0.661762,0.630789,0.00362,39.073448,0.0,9.777778,4183.666504,38780000000000,1,1,342.0,"""15dbc929"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
95,0.039419,-0.027373,-0.998735,0.001186,-87.215622,0.0,8.0,4175.0,39235000000000,1,1,342.0,"""15dbc929"""
96,0.03938,-0.027412,-0.998657,0.001111,-87.21833,0.0,7.5,4175.0,39240000000000,1,1,342.0,"""15dbc929"""
97,0.039213,-0.027633,-0.998866,0.001283,-87.226646,0.0,7.0,4175.0,39245000000000,1,1,342.0,"""15dbc929"""
